Note: This notebook is inspired from the kaggle notebook at: 
https://www.kaggle.com/code/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train?kernelSessionId=52422980

In [ ]:
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import GroupKFold
import logging
import sys
from ultralytics import YOLO
import albumentations as A
import random
# from ultralytics.engine.callbacks import Callbacks
import shutil
from tqdm import tqdm
import cv2
import random

In [ ]:
base = Path('/root')  #Path("/mnt/shared_dataset")
root = base / "YOLO"
dicom_dir = base / "physionet.org/files/vindr-cxr/1.0.0/train"
dicom_test_dir = base / "physionet.org/files/vindr-cxr/1.0.0/test"

png_dir = root / "images"
label_dir = root / "labels"
test_dir = png_dir / "test"
yaml_path = root / "yolov12/my-yolov12.yaml"

In [ ]:
for sub in ['train', 'val']:
    os.makedirs(os.path.join(png_dir,sub), exist_ok=True)
    os.makedirs(os.path.join(label_dir, sub), exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
csv_path = root / "yolo_aggregated.csv"
df = pd.read_csv(csv_path)
df["image_path"] = df["image_id"].apply(lambda x: f"YOLO/images/train/{x}.png")  # adjust path/format

In [ ]:
png_train = png_dir/"train"
actual_pngs = {p.stem for p in (png_train).glob("*.png")}

# Compare with image IDs in dataframe (after No finding drop)
df_ids = set(df["image_id"])
missing_png_ids = df_ids - actual_pngs
log_dir = "log_dir"
log_file = os.path.join(log_dir, "missing_from_disk_after_drop.txt")

if missing_png_ids:
    os.makedirs(log_dir, exist_ok=True)
    if not os.path.exists(log_file):
        with open(log_file, "w") as f:
            for mid in sorted(missing_png_ids):
                f.write(mid + "\n")
    logging.warning(f"{len(missing_png_ids)} image_ids missing as PNGs on disk.")

In [ ]:
df['class_id'] = df['class_name'].astype('category').cat.codes
class_map = dict(enumerate(df['class_name'].astype('category').cat.categories))

In [ ]:
gkf = GroupKFold(n_splits=5)
df['fold'] = -1
df = df.reset_index(drop=True)

for fold, (_, val_idx) in enumerate(gkf.split(df, groups=df['image_id'])):
    df.loc[val_idx, 'fold'] = fold

val_fold = 0  # 20% validation (1 out of 5 folds)

df_ids = set(df["image_id"])
train_ids = set(df[df['fold'] != val_fold]['image_id'].apply(lambda x: Path(x).stem))
val_ids = set(df[df['fold'] == val_fold]['image_id'].apply(lambda x: Path(x).stem))

In [ ]:
print(f"Current directory: {os.getcwd()}")
print(f"(Labels) LEN OF TRAIN DIR: {len(os.listdir('../labels/train'))}  |  LEN OF VAL DIR: {len(os.listdir('../labels/val'))}")
print(f"📊 TRAIN SIZE: {len(train_ids)} | VAL SIZE: {len(val_ids)}", file=sys.stderr)

missing = []
for name in val_ids:
    path = os.path.join((label_dir/"val"), f"{name}.txt")
    if not os.path.exists(path):
        missing.append(name)

# === Result
if not missing:
    print("✅ All .txt files exist.")
else:
    print(f"❌ Missing {len(missing)} .txt files:")
    for m in missing:
        print("  ", m)
# === Log ===
summary_path = "log_dir/train_val_summary.txt"
with open(summary_path, "w") as f:
    f.write(f"TRAIN SIZE: {len(train_ids)}\nVAL SIZE: {len(val_ids)}\n")

print(df.head(10))

In [ ]:
yaml_content = f"""# Lung Disease Dataset
path: {root}
train: images/train
val: images/val
test: images/test
nc: {len(class_map)}
names: {list(class_map.values())}
"""
print(yaml_content)
with open(yaml_path, "w") as f:
    f.write(yaml_content)

In [ ]:
run = Path(f"{root}/yolov12/runs/detect/latest")

In [ ]:
model = YOLO("yolo12m.pt")
# print(f"1 using {model}")
results = model.train(data = yaml_path, epochs=10, imgsz=640, batch = 16 ,device='cpu' , save = True, cache = True, project=run ,name="train") # callbacks = callbacks,
print(f"2 done training. Results saved in: {results.save_dir}")

In [ ]:
last_run = run / "train"
weights_path = last_run / "weights" / "best.pt"
if not weights_path.exists():
    raise FileNotFoundError(f"❌ Weights not found at {weights_path}")

print(f"🧪 Testing first 100 .png images from: {weights_path}")

test_images = sorted(test_dir.glob("*.png"))[:100]  # replace with your real test_dir
model = YOLO(str(weights_path))
results = model(
    [str(p) for p in test_images],
    save=True,
    conf=0.25,
    save_dir=f"{run}/first_100_inference"
)
